In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from collections import defaultdict  
import math  
from sys import stdout
import pickle
from evaulation import evaluate

In [2]:
def get_predict(df, pred_col, top_fill, ranknum):  
    top_fill = [int(t) for t in top_fill.split(',')]  
    scores = [-1 * i for i in range(1, len(top_fill) + 1)]  
    ids = list(df['user_id'].unique())  
    fill_df = pd.DataFrame(ids * len(top_fill), columns=['user_id'])  
    fill_df.sort_values('user_id', inplace=True)  
    fill_df['item_id'] = top_fill * len(ids)  
    fill_df[pred_col] = scores * len(ids)  
    df = df.append(fill_df)  
    df.sort_values(pred_col, ascending=False, inplace=True)  
    df = df.drop_duplicates(subset=['user_id', 'item_id'], keep='first')  
    df['rank'] = df.groupby('user_id')[pred_col].rank(method='first', ascending=False)  
    df = df[df['rank'] <= ranknum]  
    df = df.groupby('user_id')['item_id'].apply(lambda x: ','.join([str(i) for i in x])).str.split(',', expand=True).reset_index()  
    return df 

In [3]:
def merge_label(train, label):
    tmp = pd.merge(left = train,
            right = label[['user_id','item_id','future_click']],
            how = 'left',
            on = ['user_id','item_id'])
    tmp.loc[~pd.isna(tmp['future_click']), 'future_click'] = 1
    tmp.loc[pd.isna(tmp['future_click']), 'future_click'] = 0
    return tmp

In [4]:
model1_train = pd.read_csv('../../user_data/model_1/new_recall/recall_0531_addsim_addAA_RA_additemtime_addcount_addnn_addtxt_interactive_countdetail_userfeature_partialsim_emergency.csv')
model1_label = pd.read_csv('../../user_data/model_1/model_1_debias_track_answer.csv', 
                           names = ['phase','user_id','item_id','future_click'])
model1_train = merge_label(model1_train, model1_label)

In [5]:
model1_train.shape

(15397325, 71)

In [6]:
offline_train = pd.read_csv('../../user_data/offline/new_recall/recall_0531_addsim_addAA_RA_additemtime_addcount_addnn_addtxt_interactive_countdetail_userfeature_partialsim_emergency.csv')
offline_label = pd.read_csv('../../user_data/offline/offline_debias_track_answer.csv', 
                            names = ['phase','user_id','item_id','future_click'])
offline_train = merge_label(offline_train, offline_label)

In [7]:
offline_train.shape

(16405037, 71)

In [8]:
col_sel = [x for x in offline_train.columns if x not in ['user_item_count_max_time','user_item_count_min_time',
                                                        'time_interval','item_count_4h','phrase','item_count_6h',
                                                        'is_user_count_climax','item_count_2h','is_user_count_lowerpoint',
                                                        'item_count_1h']]

In [9]:
len(col_sel)

61

# 线下

In [10]:
now_phase = 9
train_path = '../../user_data/dataset/'  
test_path = '../../user_data/dataset/'
header = 'underexpose'


item_sim_list = []
item_cnt_list = []
user_item = []

whole_click = pd.DataFrame()  
for c in range(7,now_phase + 1):  
    print('phase:', c)  
    click_train = pd.read_csv(train_path + header + '_train_click_{}_time.csv'.format(c))  
    click_test = pd.read_csv(test_path + header + '_test_click_{}_time.csv'.format(c))  
    click_query = pd.read_csv(test_path + header + '_test_qtime_{}_time.csv'.format(c)) 



    all_click = click_train.append(click_test)  
    whole_click = whole_click.append(all_click)  


    whole_click =  whole_click.drop_duplicates(subset=['user_id','item_id','time'],keep='last')
    whole_click = whole_click.sort_values('time')
    whole_click = whole_click.reset_index(drop=True)

# find most popular items  
top50_click = whole_click['item_id'].value_counts().index[:500].values  
top50_click = ','.join([str(i) for i in top50_click])  

phase: 7
phase: 8
phase: 9


In [11]:
model_train = pd.concat([model1_train,offline_train])
model_train = model_train.reset_index(drop=True)

model_train_p = model_train[model_train['future_click']==1]
model_train_p = model_train_p.reset_index(drop=True)

model_train_n = model_train[model_train['future_click']==0]
model_train_n = model_train_n.reset_index(drop=True)

In [12]:
model_train_p.shape

(10307, 71)

In [13]:
online_train = pd.read_csv('../../user_data/dataset/new_recall/recall_0531_addsim_addAA_RA_additemtime_addcount_addnn_addtxt_interactive_countdetail_userfeature_partialsim_emergency.csv')

In [14]:
online_train.shape

(17383059, 70)

In [15]:
online_train = online_train[online_train['phrase']>6]
online_train = online_train.reset_index(drop=True)

In [16]:
import random

def generateDataset(df_n,df_p,random_seed):
    random.seed(random_seed)
    n_index = random.sample(list(range(len(df_n))), len(df_p)*5)
    df_ns = df_n.loc[n_index]
    df = pd.concat([df_ns,df_p])
    df = df.reset_index(drop=True)
    return df

model_train_s_1 = generateDataset(model_train_n,model_train_p,2020)
model_train_s_2 = generateDataset(model_train_n,model_train_p,0)
model_train_s_3 = generateDataset(model_train_n,model_train_p,2019)
model_train_s_4 = generateDataset(model_train_n,model_train_p,1000)
model_train_s_5 = generateDataset(model_train_n,model_train_p,3000)
model_train_s_6 = generateDataset(model_train_n,model_train_p,2021)

In [17]:
def addWeightForDataSet(df,item_degree_median,weight):
    df['sample_weight'] = df['count']/item_degree_median
    df['sample_weight'] = df['sample_weight'].apply(lambda x: 5 if x<1 else 1)
    df.loc[(df['count']<item_degree_median)&(df['future_click']==1),'sample_weight'] = weight
    df.loc[(df['count']<item_degree_median)&(df['future_click']==1)&(df['phrase'].isin([7,8,9])), 'sample_weight'] = weight * 2
    return df

In [18]:
# def addWeightForDataSet(df,item_degree_median,weight):
#     df['sample_weight'] = df['count']/item_degree_median
#     df['sample_weight'] = df['sample_weight'].apply(lambda x: 5 if x<1 else 1)
#     df.loc[(df['count']<item_degree_median)&(df['future_click']==1),'sample_weight'] = weight
#     return df

In [19]:
model_train_s_1 = addWeightForDataSet(model_train_s_1,30,35)
model_train_s_2 = addWeightForDataSet(model_train_s_2,30,35)
model_train_s_3 = addWeightForDataSet(model_train_s_3,30,35)
model_train_s_4 = addWeightForDataSet(model_train_s_4,30,35)
model_train_s_5 = addWeightForDataSet(model_train_s_5,30,35)
model_train_s_6 = addWeightForDataSet(model_train_s_6,30,35)

In [20]:
feature_list = [x for x in col_sel if x not in ['user_id','item_id','future_click','sample_weight'] 
                and 'result' not in x]

In [21]:
len(feature_list)

58

In [22]:
feature_list

['sim',
 'feature_0',
 'feature_1',
 'feature_2',
 'feature_3',
 'feature_4',
 'feature_5',
 'feature_6',
 'feature_7',
 'feature_8',
 'feature_9',
 'feature_10',
 'cn_sim',
 'hpi_sim',
 'hdi_sim',
 'lhn1_sim',
 'ra_sim',
 'aa_sim',
 'item_count_12h',
 'count',
 'diff_from_last',
 'diff_from_next',
 'diff_from_max',
 'nn',
 'txt_feature_0',
 'txt_feature_1',
 'txt_feature_2',
 'txt_feature_3',
 'txt_feature_4',
 'img_feature_0',
 'img_feature_1',
 'img_feature_2',
 'img_feature_3',
 'img_feature_4',
 'item_count',
 'user_item_count_cnt',
 'user_item_count_ratio',
 'user_item_count_timedelta',
 'item_max_time_in_phrase',
 'is_climix',
 'is_lowest_point',
 'item_diff_from_min',
 'item_diff_from_mean',
 'item_count_rankin_phrase',
 'item_count_rankin_history',
 'item_never_in_phrase',
 'user_mean_count',
 'user_max_count',
 'user_min_count',
 'sim_partial',
 'feature_0_partial',
 'feature_1_partial',
 'feature_2_partial',
 'feature_3_partial',
 'emergency_feature_0',
 'emergency_feature_1

In [23]:
feature_list_noleak = [x for x in col_sel if x not in ['user_id','item_id','future_click','sample_weight','result',
                                                       'diff_from_next'] and 'result' not in x]

In [24]:
len(feature_list_noleak)

57

In [25]:
def cbt_model(m,df_train,df_test,feat):
    m.fit(df_train[feat],df_train[['future_click']],sample_weight=list(df_train['sample_weight']))
    print(sorted(dict(zip(m.feature_names_,m.feature_importances_)).items(), key=lambda x:x[1], reverse=True))
    result = m.predict_proba(df_test[feat])[:,1]
    return result

In [26]:
df_res = pd.DataFrame()

In [27]:
import catboost as cat
clf_cbt = cat.CatBoostClassifier(iterations=2500,learning_rate=0.01,depth=6,
                                   verbose=True,thread_count=12,colsample_bylevel=0.8
                                   ,l2_leaf_reg=1
                                   ,random_seed=1024)

df_res['result_1'] = cbt_model(clf_cbt,model_train_s_1,online_train,feature_list)

df_res['result_2'] = cbt_model(clf_cbt,model_train_s_2,online_train,feature_list)

df_res['result_3'] = cbt_model(clf_cbt,model_train_s_3,online_train,feature_list)

df_res['result_4'] = cbt_model(clf_cbt,model_train_s_4,online_train,feature_list)

df_res['result_5'] = cbt_model(clf_cbt,model_train_s_5,online_train,feature_list)

df_res['result_6'] = cbt_model(clf_cbt,model_train_s_6,online_train,feature_list)


0:	learn: 0.6805088	total: 67.9ms	remaining: 2m 49s
1:	learn: 0.6664444	total: 80ms	remaining: 1m 39s
2:	learn: 0.6519928	total: 91.9ms	remaining: 1m 16s
3:	learn: 0.6397707	total: 104ms	remaining: 1m 4s
4:	learn: 0.6292102	total: 115ms	remaining: 57.6s
5:	learn: 0.6171214	total: 128ms	remaining: 53s
6:	learn: 0.6044643	total: 140ms	remaining: 49.8s
7:	learn: 0.5941775	total: 151ms	remaining: 47.1s
8:	learn: 0.5839345	total: 163ms	remaining: 45.1s
9:	learn: 0.5740598	total: 174ms	remaining: 43.4s
10:	learn: 0.5628786	total: 192ms	remaining: 43.4s
11:	learn: 0.5544541	total: 208ms	remaining: 43.1s
12:	learn: 0.5472319	total: 224ms	remaining: 42.8s
13:	learn: 0.5392422	total: 239ms	remaining: 42.4s
14:	learn: 0.5295106	total: 254ms	remaining: 42.1s
15:	learn: 0.5209826	total: 269ms	remaining: 41.7s
16:	learn: 0.5137453	total: 283ms	remaining: 41.3s
17:	learn: 0.5059572	total: 298ms	remaining: 41s
18:	learn: 0.4989656	total: 312ms	remaining: 40.7s
19:	learn: 0.4922759	total: 327ms	remaini

164:	learn: 0.2651627	total: 2.34s	remaining: 33.1s
165:	learn: 0.2648732	total: 2.35s	remaining: 33.1s
166:	learn: 0.2645205	total: 2.37s	remaining: 33.1s
167:	learn: 0.2641204	total: 2.38s	remaining: 33s
168:	learn: 0.2636765	total: 2.39s	remaining: 33s
169:	learn: 0.2633655	total: 2.41s	remaining: 33s
170:	learn: 0.2630312	total: 2.42s	remaining: 33s
171:	learn: 0.2627084	total: 2.45s	remaining: 33.1s
172:	learn: 0.2624156	total: 2.47s	remaining: 33.2s
173:	learn: 0.2621330	total: 2.49s	remaining: 33.3s
174:	learn: 0.2618070	total: 2.5s	remaining: 33.3s
175:	learn: 0.2615520	total: 2.52s	remaining: 33.3s
176:	learn: 0.2612632	total: 2.54s	remaining: 33.3s
177:	learn: 0.2609534	total: 2.55s	remaining: 33.3s
178:	learn: 0.2607107	total: 2.57s	remaining: 33.3s
179:	learn: 0.2604100	total: 2.58s	remaining: 33.3s
180:	learn: 0.2601564	total: 2.6s	remaining: 33.3s
181:	learn: 0.2599313	total: 2.62s	remaining: 33.3s
182:	learn: 0.2596693	total: 2.63s	remaining: 33.3s
183:	learn: 0.2593823	

329:	learn: 0.2380296	total: 4.87s	remaining: 32s
330:	learn: 0.2379650	total: 4.88s	remaining: 32s
331:	learn: 0.2379091	total: 4.89s	remaining: 32s
332:	learn: 0.2378119	total: 4.91s	remaining: 32s
333:	learn: 0.2376754	total: 4.92s	remaining: 31.9s
334:	learn: 0.2375096	total: 4.94s	remaining: 31.9s
335:	learn: 0.2374572	total: 4.96s	remaining: 31.9s
336:	learn: 0.2374008	total: 4.97s	remaining: 31.9s
337:	learn: 0.2373542	total: 4.99s	remaining: 31.9s
338:	learn: 0.2372824	total: 5s	remaining: 31.9s
339:	learn: 0.2371865	total: 5.02s	remaining: 31.9s
340:	learn: 0.2370727	total: 5.03s	remaining: 31.9s
341:	learn: 0.2370027	total: 5.05s	remaining: 31.8s
342:	learn: 0.2368947	total: 5.06s	remaining: 31.8s
343:	learn: 0.2368209	total: 5.08s	remaining: 31.8s
344:	learn: 0.2367496	total: 5.09s	remaining: 31.8s
345:	learn: 0.2366887	total: 5.11s	remaining: 31.8s
346:	learn: 0.2366005	total: 5.13s	remaining: 31.8s
347:	learn: 0.2365248	total: 5.14s	remaining: 31.8s
348:	learn: 0.2364233	t

497:	learn: 0.2274629	total: 7.41s	remaining: 29.8s
498:	learn: 0.2274293	total: 7.43s	remaining: 29.8s
499:	learn: 0.2273702	total: 7.44s	remaining: 29.8s
500:	learn: 0.2273237	total: 7.46s	remaining: 29.8s
501:	learn: 0.2272471	total: 7.47s	remaining: 29.7s
502:	learn: 0.2272047	total: 7.49s	remaining: 29.7s
503:	learn: 0.2271929	total: 7.5s	remaining: 29.7s
504:	learn: 0.2271266	total: 7.52s	remaining: 29.7s
505:	learn: 0.2271084	total: 7.53s	remaining: 29.7s
506:	learn: 0.2270312	total: 7.55s	remaining: 29.7s
507:	learn: 0.2269912	total: 7.56s	remaining: 29.7s
508:	learn: 0.2269688	total: 7.58s	remaining: 29.6s
509:	learn: 0.2269362	total: 7.59s	remaining: 29.6s
510:	learn: 0.2268809	total: 7.61s	remaining: 29.6s
511:	learn: 0.2268509	total: 7.62s	remaining: 29.6s
512:	learn: 0.2267965	total: 7.64s	remaining: 29.6s
513:	learn: 0.2267410	total: 7.65s	remaining: 29.6s
514:	learn: 0.2266951	total: 7.67s	remaining: 29.6s
515:	learn: 0.2266397	total: 7.68s	remaining: 29.5s
516:	learn: 0

665:	learn: 0.2209064	total: 9.95s	remaining: 27.4s
666:	learn: 0.2208560	total: 9.96s	remaining: 27.4s
667:	learn: 0.2207923	total: 9.98s	remaining: 27.4s
668:	learn: 0.2207797	total: 9.99s	remaining: 27.4s
669:	learn: 0.2207655	total: 10s	remaining: 27.3s
670:	learn: 0.2207498	total: 10s	remaining: 27.3s
671:	learn: 0.2207221	total: 10s	remaining: 27.3s
672:	learn: 0.2206948	total: 10.1s	remaining: 27.3s
673:	learn: 0.2206655	total: 10.1s	remaining: 27.3s
674:	learn: 0.2206396	total: 10.1s	remaining: 27.3s
675:	learn: 0.2206289	total: 10.1s	remaining: 27.2s
676:	learn: 0.2205900	total: 10.1s	remaining: 27.2s
677:	learn: 0.2205700	total: 10.1s	remaining: 27.2s
678:	learn: 0.2205402	total: 10.1s	remaining: 27.2s
679:	learn: 0.2205250	total: 10.2s	remaining: 27.2s
680:	learn: 0.2204932	total: 10.2s	remaining: 27.2s
681:	learn: 0.2204697	total: 10.2s	remaining: 27.2s
682:	learn: 0.2204305	total: 10.2s	remaining: 27.1s
683:	learn: 0.2204095	total: 10.2s	remaining: 27.1s
684:	learn: 0.2203

...

1827:	learn: 0.1775504	total: 25.3s	remaining: 9.31s
1828:	learn: 0.1775168	total: 25.3s	remaining: 9.3s
1829:	learn: 0.1774868	total: 25.4s	remaining: 9.29s
1830:	learn: 0.1774502	total: 25.4s	remaining: 9.27s
1831:	learn: 0.1774168	total: 25.4s	remaining: 9.26s
1832:	learn: 0.1773909	total: 25.4s	remaining: 9.24s
1833:	learn: 0.1773477	total: 25.4s	remaining: 9.23s
1834:	learn: 0.1773094	total: 25.4s	remaining: 9.22s
1835:	learn: 0.1772802	total: 25.4s	remaining: 9.2s
1836:	learn: 0.1772483	total: 25.5s	remaining: 9.19s
1837:	learn: 0.1772199	total: 25.5s	remaining: 9.17s
1838:	learn: 0.1771898	total: 25.5s	remaining: 9.16s
1839:	learn: 0.1771570	total: 25.5s	remaining: 9.15s
1840:	learn: 0.1771149	total: 25.5s	remaining: 9.13s
1841:	learn: 0.1770853	total: 25.5s	remaining: 9.12s
1842:	learn: 0.1770613	total: 25.5s	remaining: 9.11s
1843:	learn: 0.1770248	total: 25.6s	remaining: 9.09s
1844:	learn: 0.1769884	total: 25.6s	remaining: 9.08s
1845:	learn: 0.1769652	total: 25.6s	remaining: 9

1992:	learn: 0.1721573	total: 27.6s	remaining: 7.02s
1993:	learn: 0.1721243	total: 27.6s	remaining: 7.01s
1994:	learn: 0.1720981	total: 27.6s	remaining: 7s
1995:	learn: 0.1720698	total: 27.6s	remaining: 6.98s
1996:	learn: 0.1720365	total: 27.7s	remaining: 6.97s
1997:	learn: 0.1719957	total: 27.7s	remaining: 6.95s
1998:	learn: 0.1719618	total: 27.7s	remaining: 6.94s
1999:	learn: 0.1719375	total: 27.7s	remaining: 6.92s
2000:	learn: 0.1719022	total: 27.7s	remaining: 6.91s
2001:	learn: 0.1718675	total: 27.7s	remaining: 6.9s
2002:	learn: 0.1718429	total: 27.7s	remaining: 6.88s
2003:	learn: 0.1718198	total: 27.8s	remaining: 6.87s
2004:	learn: 0.1717862	total: 27.8s	remaining: 6.86s
2005:	learn: 0.1717529	total: 27.8s	remaining: 6.84s
2006:	learn: 0.1717194	total: 27.8s	remaining: 6.83s
2007:	learn: 0.1716891	total: 27.8s	remaining: 6.81s
2008:	learn: 0.1716591	total: 27.8s	remaining: 6.8s
2009:	learn: 0.1716232	total: 27.8s	remaining: 6.79s
2010:	learn: 0.1715911	total: 27.9s	remaining: 6.77

2157:	learn: 0.1670325	total: 29.9s	remaining: 4.73s
2158:	learn: 0.1670050	total: 29.9s	remaining: 4.72s
2159:	learn: 0.1669874	total: 29.9s	remaining: 4.7s
2160:	learn: 0.1669684	total: 29.9s	remaining: 4.69s
2161:	learn: 0.1669506	total: 29.9s	remaining: 4.68s
2162:	learn: 0.1669329	total: 29.9s	remaining: 4.66s
2163:	learn: 0.1669097	total: 29.9s	remaining: 4.65s
2164:	learn: 0.1668860	total: 30s	remaining: 4.63s
2165:	learn: 0.1668477	total: 30s	remaining: 4.62s
2166:	learn: 0.1668134	total: 30s	remaining: 4.61s
2167:	learn: 0.1667744	total: 30s	remaining: 4.59s
2168:	learn: 0.1667503	total: 30s	remaining: 4.58s
2169:	learn: 0.1667231	total: 30s	remaining: 4.57s
2170:	learn: 0.1666947	total: 30s	remaining: 4.55s
2171:	learn: 0.1666665	total: 30s	remaining: 4.54s
2172:	learn: 0.1666172	total: 30.1s	remaining: 4.52s
2173:	learn: 0.1665790	total: 30.1s	remaining: 4.51s
2174:	learn: 0.1665485	total: 30.1s	remaining: 4.5s
2175:	learn: 0.1665192	total: 30.1s	remaining: 4.48s
2176:	learn

2322:	learn: 0.1624427	total: 32.1s	remaining: 2.45s
2323:	learn: 0.1624163	total: 32.1s	remaining: 2.43s
2324:	learn: 0.1623944	total: 32.1s	remaining: 2.42s
2325:	learn: 0.1623637	total: 32.1s	remaining: 2.4s
2326:	learn: 0.1623445	total: 32.2s	remaining: 2.39s
2327:	learn: 0.1623094	total: 32.2s	remaining: 2.38s
2328:	learn: 0.1622856	total: 32.2s	remaining: 2.36s
2329:	learn: 0.1622520	total: 32.2s	remaining: 2.35s
2330:	learn: 0.1622313	total: 32.2s	remaining: 2.33s
2331:	learn: 0.1622064	total: 32.2s	remaining: 2.32s
2332:	learn: 0.1621787	total: 32.2s	remaining: 2.31s
2333:	learn: 0.1621445	total: 32.3s	remaining: 2.29s
2334:	learn: 0.1621258	total: 32.3s	remaining: 2.28s
2335:	learn: 0.1620887	total: 32.3s	remaining: 2.27s
2336:	learn: 0.1620587	total: 32.3s	remaining: 2.25s
2337:	learn: 0.1620326	total: 32.3s	remaining: 2.24s
2338:	learn: 0.1620091	total: 32.3s	remaining: 2.22s
2339:	learn: 0.1619813	total: 32.3s	remaining: 2.21s
2340:	learn: 0.1619531	total: 32.3s	remaining: 

2487:	learn: 0.1580644	total: 34.3s	remaining: 166ms
2488:	learn: 0.1580255	total: 34.4s	remaining: 152ms
2489:	learn: 0.1579987	total: 34.4s	remaining: 138ms
2490:	learn: 0.1579716	total: 34.4s	remaining: 124ms
2491:	learn: 0.1579320	total: 34.4s	remaining: 110ms
2492:	learn: 0.1579182	total: 34.4s	remaining: 96.6ms
2493:	learn: 0.1578918	total: 34.4s	remaining: 82.8ms
2494:	learn: 0.1578680	total: 34.4s	remaining: 69ms
2495:	learn: 0.1578439	total: 34.4s	remaining: 55.2ms
2496:	learn: 0.1578071	total: 34.5s	remaining: 41.4ms
2497:	learn: 0.1577864	total: 34.5s	remaining: 27.6ms
2498:	learn: 0.1577688	total: 34.5s	remaining: 13.8ms
2499:	learn: 0.1577399	total: 34.5s	remaining: 0us
[('is_lowest_point', 10.601281850959383), ('is_climix', 8.138511152192839), ('item_count_rankin_phrase', 6.35751605932988), ('item_count', 5.800499956567959), ('item_never_in_phrase', 5.272948930388485), ('feature_9', 4.557571243070302), ('nn', 4.4659682755894945), ('item_diff_from_mean', 4.162379094531647)

In [28]:
df_res['phrase'] = online_train['phrase']
df_res['user_id'] = online_train['user_id']
df_res['item_id'] = online_train['item_id']

In [29]:
df_res['result'] = df_res['result_1'] + \
df_res['result_2'] + \
df_res['result_3'] + \
df_res['result_4'] + \
df_res['result_5'] + \
df_res['result_6'] 

In [30]:
def lgb_model(df_train,df_test,feat,params,num_round):
    train_data = lgb.Dataset(df_train[feat], 
                         label=df_train[['future_click']],weight=df_train['sample_weight'])  
    print('lgb training')
    bst = lgb.train(params,
                train_data,
                num_round)    
    print('lgb predicting')
    result = bst.predict(df_test[feat])    
    return result

In [31]:
import lightgbm as lgb
import time

num_round = 2500
params = {
        'learning_rate': 0.01,
        'boosting_type': 'dart',
        'objective': 'binary',
        #'metric': 'auc',
        'max_depth': 6,
        'feature_fraction': 0.8,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'seed': 1,
        'bagging_seed': 10,
        'feature_fraction_seed': 7,
        'min_data_in_leaf': 20,
        'nthread': 8,
        'verbose': 1,
    }

In [32]:
df_res['lgb_dart_1'] = lgb_model(model_train_s_1,online_train,feature_list,params,num_round)

df_res['lgb_dart_2'] = lgb_model(model_train_s_2,online_train,feature_list,params,num_round)

df_res['lgb_dart_3'] = lgb_model(model_train_s_3,online_train,feature_list,params,num_round)

df_res['lgb_dart_4'] = lgb_model(model_train_s_4,online_train,feature_list,params,num_round)

df_res['lgb_dart_5'] = lgb_model(model_train_s_5,online_train,feature_list,params,num_round)

df_res['lgb_dart_6'] = lgb_model(model_train_s_6,online_train,feature_list,params,num_round)


lgb training
lgb predicting
lgb training
lgb predicting
lgb training
lgb predicting
lgb training
lgb predicting
lgb training
lgb predicting
lgb training
lgb predicting


In [33]:
df_res['result_lgb_dart'] = df_res['lgb_dart_1'] + \
df_res['lgb_dart_2'] + \
df_res['lgb_dart_3'] + \
df_res['lgb_dart_4'] + \
df_res['lgb_dart_5'] + \
df_res['lgb_dart_6'] 

In [34]:
df_res['result'] = df_res['result']/6
df_res['result_lgb_dart'] = df_res['result_lgb_dart']/6

In [35]:
df_res['count_na'] = online_train['count'].apply(lambda x: np.nan if x ==0 else x)
df_res['m'] = df_res['count_na'].apply(lambda x:max(0.61,1/math.log1p(x+1)))

In [36]:
df_res['result_PostProcess'] = df_res['result'] * df_res['m']
df_res['result_lgb_dart_PostProcess'] = df_res['result_lgb_dart'] * df_res['m']

In [37]:
df_res['ensemble1'] = 10 / ( 6/df_res['result_PostProcess'] + 4/df_res['result_lgb_dart_PostProcess']) 
df_res['ensemble2'] = np.power( df_res['result_PostProcess']**6 * df_res['result_lgb_dart_PostProcess']**4 , 1/10) 

In [38]:
df_res['ensemble'] = df_res['ensemble1']  + df_res['ensemble2'] 

In [39]:
#df_res['PostProcess'] = df_res['ensemble'] * df_res['m']

recom_df = df_res[['user_id','item_id','ensemble']]
result = get_predict(recom_df, 'ensemble', top50_click, 50) 
result['user_id'] = result['user_id'].astype(int)
result.to_csv('../../prediction_result/prediction_result.csv', index=False, header=None)

